This is MLP method

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random

In [2]:
# Data pre-processing
file_path='total_new.csv'
df=pd.read_csv(file_path)
df["Metadata_gene"]=df["Metadata_gene"].fillna('empty')
df_fill_empty=df
df_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']=='negcon']
df_rm_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']!='negcon']
df_rm_empty_gene=df_fill_empty[df_fill_empty['Metadata_gene']!='empty']
df_rm_orf=df_rm_empty_gene[df_rm_empty_gene["Metadata_experiment_type"]!='ORF']

# sampling
df_proceed=df_rm_orf[~df_rm_orf['Metadata_pert_type'].isin(['control'])]
gene_num=(df_proceed['Metadata_gene'].value_counts())
gene_num_list=gene_num.index.tolist()

# Only in list of plate is for mAP classifiation, so we filter it out
lst=['BR00116991','BR00116992','BR00116993','BR00116994','BR00117015','BR00117016','BR00117017','BR00117019','BR00116995','BR00117024','BR00117025','BR00117026','BR00117010','BR00117011','BR00117012','BR00117013']

# 8:2 Split Target Gene
random.seed(1)
# 104 gene for training
train_lst=random.sample(gene_num_list, 104)#####
# 26 gene for testing
test_lst = []
for m in gene_num_list:
    if m not in train_lst:
        test_lst.append(m)

# filter out compound        
df_compound=df_proceed[df_proceed['Metadata_experiment_type']=='Compound']

# 80% target gene of Compound + Gene perturbation 
dftrain1=df_compound[df_compound['Metadata_gene'].isin(train_lst)]  

# 100% CRISPR for Train
dftrain2=df_proceed[df_proceed['Metadata_experiment_type']=='CRISPR']    

# To meet the mAP evaluation Plate
# 2x2 : Time_delay x Cell_line
train_part_total=pd.DataFrame()
for i in lst:
    train_part_total=pd.concat([train_part_total,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
train_part0=pd.DataFrame()
for i in ['BR00116991','BR00116992','BR00116993','BR00116994']:
    train_part0=pd.concat([train_part0,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part1=pd.DataFrame()
for i in ['BR00117015','BR00117016','BR00117017','BR00117019']:
    train_part1=pd.concat([train_part1,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part2=pd.DataFrame()
for i in ['BR00116995','BR00117024','BR00117025','BR00117026']:
    train_part2=pd.concat([train_part2,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part3=pd.DataFrame()
for i in ['BR00117010','BR00117011','BR00117012','BR00117013']:
    train_part3=pd.concat([train_part3,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])  
    
# test data sampling
gene_seen=pd.DataFrame()
for i in train_lst:
    train_part0_filtered=train_part0.loc[train_part0['Metadata_gene']==i,:]
    train_part1_filtered=train_part1.loc[train_part1['Metadata_gene']==i,:]
    train_part2_filtered=train_part2.loc[train_part2['Metadata_gene']==i,:]
    train_part3_filtered=train_part3.loc[train_part3['Metadata_gene']==i,:]

# To filter seen/unseen gene from compound perturbation

    if len(train_part0_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part0_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part0_filtered[train_part0_filtered['Metadata_pert_iname']==filter_out]])
        
    if len(train_part1_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part1_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part1_filtered[train_part1_filtered['Metadata_pert_iname']==filter_out]]) 
        
    if len(train_part2_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part2_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part2_filtered[train_part2_filtered['Metadata_pert_iname']==filter_out]])  
        
    if len(train_part3_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part3_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part3_filtered[train_part3_filtered['Metadata_pert_iname']==filter_out]])

train_part_total_final=train_part_total.drop(index=list(gene_seen['Unnamed: 0'].index))



# SEEN test
df_test1=pd.concat([train_part_total,dftrain2,df_negcon]) 

# Train 
df_train=pd.concat([train_part_total_final,dftrain2]) 

### Unseen test
df_test2=df_proceed[df_proceed['Metadata_gene'].isin(test_lst)]
df_test2=pd.concat([df_test2,df_negcon,dftrain2])
df_test2=df_test2.drop_duplicates()


# Train_test pre-process
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
testX=df_test1.drop('Metadata_gene', axis = 1)
y_test = df_test1['Metadata_gene']
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)

/tmp/ipykernel_1908554/4262354691.py:3: DtypeWarning: Columns (2,6,7,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


In [3]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, recall_score

# Scale the training and test sets using StandardScaler
scaler = StandardScaler()
trainX_scaled = scaler.fit_transform(trainX)
testX_scaled = scaler.transform(testX)
# Train the MLPClassifier model
mlp_clf = MLPClassifier(hidden_layer_sizes=(1280, 630, 300, 176),
                        max_iter=300, activation='relu', verbose=True, tol=1e-8,
                        solver='adam')
mlp_clf.fit(trainX_scaled, trainY)

Iteration 1, loss = 4.63280833
Iteration 2, loss = 3.81756013
Iteration 3, loss = 3.21482643
Iteration 4, loss = 2.66420170
Iteration 5, loss = 2.24928450
Iteration 6, loss = 1.80763293
Iteration 7, loss = 1.42421490
Iteration 8, loss = 1.07880366
Iteration 9, loss = 0.84882650
Iteration 10, loss = 0.60661853
Iteration 11, loss = 0.40217994
Iteration 12, loss = 0.31671142
Iteration 13, loss = 0.27985346
Iteration 14, loss = 0.27405733
Iteration 15, loss = 0.23868805
Iteration 16, loss = 0.15322566
Iteration 17, loss = 0.11322482
Iteration 18, loss = 0.08606683
Iteration 19, loss = 0.07462781
Iteration 20, loss = 0.05301533
Iteration 21, loss = 0.04058625
Iteration 22, loss = 0.03109133
Iteration 23, loss = 0.01720527
Iteration 24, loss = 0.01438768
Iteration 25, loss = 0.01479688
Iteration 26, loss = 0.01666793
Iteration 27, loss = 0.02541732
Iteration 28, loss = 0.04305475
Iteration 29, loss = 0.05624399
Iteration 30, loss = 0.04239180
Iteration 31, loss = 0.05722283
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(1280, 630, 300, 176), max_iter=300, tol=1e-08,
              verbose=True)

In [4]:
##### Feature Transformation Seen 
import torch
import torch.nn as nn
relu = nn.ReLU(inplace=True)
original_seen=df_test1.iloc[:,df_test1.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
sc=StandardScaler()
scaler = sc.fit(original_seen)
original_seen_scaled = scaler.transform(original_seen)
layer_1=np.array(relu(torch.from_numpy(np.dot(original_seen_scaled,mlp_clf.coefs_[0])+mlp_clf.intercepts_[0])))
layer_2=np.array(relu(torch.from_numpy(np.dot(layer_1,mlp_clf.coefs_[1])+mlp_clf.intercepts_[1])))
layer_3=np.array(relu(torch.from_numpy(np.dot(layer_2,mlp_clf.coefs_[2])+mlp_clf.intercepts_[2])))
transformen_seen=df_test1.iloc[:,0:df_test1.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
feature=pd.DataFrame(layer_3)
feature.index=transformen_seen.index
feature_new =pd.concat([transformen_seen,feature,df_test1['Metadata_experiment_type']], axis=1)
feature_new

,Unnamed: 0,Metadata_broad_sample,Metadata_solvent,Metadata_experiment_type,Metadata_Plate,Metadata_Well,Metadata_InChIKey,Metadata_pert_iname,Metadata_pubchem_cid,Metadata_gene,...,291,292,293,294,295,296,297,298,299,Metadata_experiment_type
12670,0,BRD-A86665761-001-01-1,DMSO,Compound,BR00116991,A01,TZDUHAJSIBHXDL-UHFFFAOYSA-N,gabapentin-enacarbil,9883900.0,CACNB4,...,0.000000,0.0,0.0,0.0,0.000000,9.278545,0.0,0.0,0.000000,Compound
12672,2,BRD-A22032524-074-09-9,DMSO,Compound,BR00116991,A03,HTIQEAQVCYTUBX-UHFFFAOYSA-N,amlodipine,2162.0,CACNA2D3,...,0.000000,0.0,0.0,0.0,0.000000,2.763155,0.0,0.0,0.815806,Compound
12673,3,BRD-A01078468-001-14-8,DMSO,Compound,BR00116991,A04,PBBGSZCBWVPOOL-UHFFFAOYSA-N,hexestrol,3606.0,AKR1C1,...,0.000000,0.0,0.0,0.0,0.033328,4.252575,0.0,0.0,0.400650,Compound
12675,5,BRD-K36574127-001-01-3,DMSO,Compound,BR00116991,A06,NYNZQNWKBKUAII-KBXCAEBGSA-N,LOXO-101,46189000.0,NTRK1,...,0.000000,0.0,0.0,0.0,0.000000,6.942296,0.0,0.0,0.000000,Compound
12676,6,BRD-K74913225-001-14-0,DMSO,Compound,BR00116991,A07,HCRKCZRJWPKOAR-JTQLQIEISA-N,brinzolamide,68844.0,CA5A,...,0.000000,0.0,0.0,0.0,0.935707,7.469378,0.0,0.0,0.000000,Compound
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19447,333,NaN,DMSO,Compound,BR00117024,N22,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,DMSO,679.0,empty,...,0.970439,0.0,0.0,0.0,0.000000,4.467202,0.0,0.0,0.000000,Compound
19456,342,NaN,DMSO,Compound,BR00117024,O07,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,DMSO,679.0,empty,...,0.725292,0.0,0.0,0.0,3.206739,0.000000,0.0,0.0,0.000000,Compound
19467,353,NaN,DMSO,Compound,BR00117024,O18,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,DMSO,679.0,empty,...,0.000000,0.0,0.0,0.0,0.000000,6.321487,0.0,0.0,0.000000,Compound
19478,364,NaN,DMSO,Compound,BR00117024,P05,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,DMSO,679.0,empty,...,0.000000,0.0,0.0,0.0,0.000000,13.884514,0.0,0.0,0.000000,Compound


In [5]:
feature_new.to_csv('MLP_seen_new.csv')
df_test1.to_csv('MLP_seen_old.csv')    

In [6]:
##### Feature Transformation Seen 
import torch
import torch.nn as nn
relu = nn.ReLU(inplace=True)
original_seen=df_test2.iloc[:,df_test2.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
sc=StandardScaler()
scaler = sc.fit(original_seen)
original_seen_scaled = scaler.transform(original_seen)
layer_1=np.array(relu(torch.from_numpy(np.dot(original_seen_scaled,mlp_clf.coefs_[0])+mlp_clf.intercepts_[0])))
layer_2=np.array(relu(torch.from_numpy(np.dot(layer_1,mlp_clf.coefs_[1])+mlp_clf.intercepts_[1])))
layer_3=np.array(relu(torch.from_numpy(np.dot(layer_2,mlp_clf.coefs_[2])+mlp_clf.intercepts_[2])))
transformen_seen=df_test2.iloc[:,0:df_test2.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
feature=pd.DataFrame(layer_3)
feature.index=transformen_seen.index
feature_new =pd.concat([transformen_seen,feature,df_test2['Metadata_experiment_type']], axis=1)

,Unnamed: 0,Metadata_broad_sample,Metadata_solvent,Metadata_experiment_type,Metadata_Plate,Metadata_Well,Metadata_InChIKey,Metadata_pert_iname,Metadata_pubchem_cid,Metadata_gene,...,291,292,293,294,295,296,297,298,299,Metadata_experiment_type
4,4,BRD-K48278478-001-01-2,DMSO,Compound,BR00117016,A05,LOUPRKONTZGTKE-AFHBHXEDSA-N,quinine,94175.0,KCNN4,...,0.000000,0.0,0.0,0.000000,0.000000,7.276139,0.0,0.0,0.000000,Compound
19,19,BRD-K91825936-001-03-9,DMSO,Compound,BR00117016,A20,XQYASZNUFDVMFH-CQSZACIVSA-N,ZK811752,512280.0,CCR1,...,0.000000,0.0,0.0,0.000000,0.000000,16.340513,0.0,0.0,0.000000,Compound
20,20,BRD-K38775274-001-22-1,DMSO,Compound,BR00117016,A21,UREBDLICKHMUKA-CXSFZGCWSA-N,dexamethasone,5743.0,ANXA1,...,0.000000,0.0,0.0,0.000000,0.000000,35.669980,0.0,0.0,0.000000,Compound
24,24,BRD-K26439554-001-01-3,DMSO,Compound,BR00117016,B01,HFYPTENHTPNXGP-UHFFFAOYSA-N,H2L-5765834,1365700.0,LPAR1,...,0.000000,0.0,0.0,4.205173,0.000000,5.969017,0.0,0.0,0.000000,Compound
30,30,BRD-K06890060-001-09-1,DMSO,Compound,BR00117016,B07,NMKJFZCBCIUYHI-UHFFFAOYSA-N,alda-1,831630.0,ALDH2,...,3.471665,0.0,0.0,0.000000,0.000000,4.595613,0.0,0.0,0.000000,Compound
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18339,377,BRDN0001054361,NaN,CRISPR,BR00118045,P18,NaN,NaN,NaN,PDE3A,...,0.723219,0.0,0.0,10.307343,0.000000,1.021840,0.0,0.0,0.000000,CRISPR
18340,378,BRDN0001481136,NaN,CRISPR,BR00118045,P19,NaN,NaN,NaN,P2RY12,...,0.000000,0.0,0.0,2.796397,0.000000,3.727521,0.0,0.0,0.000000,CRISPR
18342,380,BRDN0001484228,NaN,CRISPR,BR00118045,P21,NaN,NaN,NaN,GPR119,...,0.627181,0.0,0.0,7.800848,0.000000,0.000000,0.0,0.0,0.000000,CRISPR
18343,381,BRDN0001487618,NaN,CRISPR,BR00118045,P22,NaN,NaN,NaN,OPRM1,...,0.000000,0.0,0.0,9.726081,0.000000,0.000000,0.0,0.0,0.000000,CRISPR


In [7]:
feature_new.to_csv('MLP_unseen_new.csv')
df_test2.to_csv('MLP_unseen_old.csv')  

This is SLPP Method

In [9]:
# Data pre-processing
file_path='total_new.csv'
df=pd.read_csv(file_path)
df["Metadata_gene"]=df["Metadata_gene"].fillna('empty')
df_fill_empty=df
df_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']=='negcon']
df_rm_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']!='negcon']
df_rm_empty_gene=df_fill_empty[df_fill_empty['Metadata_gene']!='empty']
df_rm_orf=df_rm_empty_gene[df_rm_empty_gene["Metadata_experiment_type"]!='ORF']
# sampling
df_proceed=df_rm_orf[~df_rm_orf['Metadata_pert_type'].isin(['control'])]
gene_num=(df_proceed['Metadata_gene'].value_counts())
gene_num_list=gene_num.index.tolist()
# Only in list of plate is for mAP classifiation, so we filter it out
lst=['BR00116991','BR00116992','BR00116993','BR00116994','BR00117015','BR00117016','BR00117017','BR00117019','BR00116995','BR00117024','BR00117025','BR00117026','BR00117010','BR00117011','BR00117012','BR00117013']

# 8:2 Split Target Gene
random.seed(1)
train_lst=random.sample(gene_num_list, 104)#####
test_lst = []
for m in gene_num_list:
    if m not in train_lst:
        test_lst.append(m)
        
df_compound=df_proceed[df_proceed['Metadata_experiment_type']=='Compound']

# 80% target gene of Compound + Gene perturbation 

dftrain1=df_compound[df_compound['Metadata_gene'].isin(train_lst)]  

# 100% CRISPR for Train
dftrain2=df_proceed[df_proceed['Metadata_experiment_type']=='CRISPR']    

# To meet the mAP evaluation Plate
train_part_total=pd.DataFrame()
for i in lst:
    train_part_total=pd.concat([train_part_total,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
train_part0=pd.DataFrame()
for i in ['BR00116991','BR00116992','BR00116993','BR00116994']:
    train_part0=pd.concat([train_part0,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part1=pd.DataFrame()
for i in ['BR00117015','BR00117016','BR00117017','BR00117019']:
    train_part1=pd.concat([train_part1,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part2=pd.DataFrame()
for i in ['BR00116995','BR00117024','BR00117025','BR00117026']:
    train_part2=pd.concat([train_part2,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part3=pd.DataFrame()
for i in ['BR00117010','BR00117011','BR00117012','BR00117013']:
    train_part3=pd.concat([train_part3,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])  
    
# test data sampling
gene_seen=pd.DataFrame()
for i in train_lst:
    train_part0_filtered=train_part0.loc[train_part0['Metadata_gene']==i,:]
    train_part1_filtered=train_part1.loc[train_part1['Metadata_gene']==i,:]
    train_part2_filtered=train_part2.loc[train_part2['Metadata_gene']==i,:]
    train_part3_filtered=train_part3.loc[train_part3['Metadata_gene']==i,:]

# To filter seen/unseen gene from compound perturbation

    if len(train_part0_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part0_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part0_filtered[train_part0_filtered['Metadata_pert_iname']==filter_out]])
        
    if len(train_part1_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part1_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part1_filtered[train_part1_filtered['Metadata_pert_iname']==filter_out]]) 
        
    if len(train_part2_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part2_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part2_filtered[train_part2_filtered['Metadata_pert_iname']==filter_out]])  
        
    if len(train_part3_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part3_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part3_filtered[train_part3_filtered['Metadata_pert_iname']==filter_out]])

train_part_total_final=train_part_total.drop(index=list(gene_seen['Unnamed: 0'].index))



# SEEN


df_test1=pd.concat([train_part_total,dftrain2,df_negcon])  ####其中一种验证方法！！！
df_train=pd.concat([train_part_total_final,dftrain2]) ###训练
### Unseen
df_test2=df_proceed[df_proceed['Metadata_gene'].isin(test_lst)]
df_test2=pd.concat([df_test2,df_negcon,dftrain2])
df_test2=df_test2.drop_duplicates()


# Train_test pre-process
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
y_test=df_test1['Metadata_gene']
testX=df_test1.drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)

/tmp/ipykernel_1908554/557094528.py:3: DtypeWarning: Columns (2,6,7,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


In [10]:
X_train = trainX
y_train = y_train.reset_index()
y_train=y_train['Metadata_gene'].tolist()
n_components=300
sigma=1.0

In [11]:
    # Standardize the data
    X_train = StandardScaler().fit_transform(X_train)
    # Compute the pairwise distance matrix
    D = np.zeros((X_train.shape[0], X_train.shape[0]))
    for i in range(X_train.shape[0]):
        for j in range(X_train.shape[0]):
            D[i, j] = np.linalg.norm(X_train[i] - X_train[j])

    # Compute the adjacency graph using a Gaussian kernel
    W = np.zeros((X_train.shape[0], X_train.shape[0]))
    for i in range(X_train.shape[0]):
        for j in range(X_train.shape[0]):
            W[i, j] = np.exp(-D[i, j] ** 2 / (2 * sigma ** 2)) * (y_train[i] == y_train[j])

    # Compute the degree matrix
    D = np.diag(np.sum(W, axis=1))

    # Compute the Laplacian matrix
    L = D - W

    # Compute the eigenvectors
    eigvals, eigvecs = np.linalg.eig(np.dot(np.dot(X_train.T, L), X_train))

    # Sort the eigenvectors by eigenvalues in descending order
    sort_indices = np.argsort(eigvals)[::-1]
    eigvecs = eigvecs[:, sort_indices]

    # Select the first n_components eigenvectors
    eigvecs = eigvecs[:, :n_components]
    eigvecs=np.real(eigvecs)

In [12]:
## transformed seen_test_data
transformed_data = np.dot(testX, eigvecs)
new_feature=pd.DataFrame(transformed_data)
testX1=df_test1.iloc[:,:df_test1.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
testX1=testX1.reset_index()
dfnew =pd.concat([testX1,new_feature],axis=1)

,index,Unnamed: 0,Metadata_broad_sample,Metadata_solvent,Metadata_experiment_type,Metadata_Plate,Metadata_Well,Metadata_InChIKey,Metadata_pert_iname,Metadata_pubchem_cid,...,290,291,292,293,294,295,296,297,298,299
0,12670,0,BRD-A86665761-001-01-1,DMSO,Compound,BR00116991,A01,TZDUHAJSIBHXDL-UHFFFAOYSA-N,gabapentin-enacarbil,9883900.0,...,-2.840770,0.200351,1.824592,-0.839663,-0.839663,0.121115,0.121115,2.034312,-2.349212,-2.134020
1,12672,2,BRD-A22032524-074-09-9,DMSO,Compound,BR00116991,A03,HTIQEAQVCYTUBX-UHFFFAOYSA-N,amlodipine,2162.0,...,-5.076788,0.838999,2.554194,1.186119,1.186119,0.960031,0.960031,2.851644,-2.661988,-2.492771
2,12673,3,BRD-A01078468-001-14-8,DMSO,Compound,BR00116991,A04,PBBGSZCBWVPOOL-UHFFFAOYSA-N,hexestrol,3606.0,...,-6.156763,0.444105,2.792689,1.079891,1.079891,1.224905,1.224905,3.309753,-4.715892,-4.667970
3,12675,5,BRD-K36574127-001-01-3,DMSO,Compound,BR00116991,A06,NYNZQNWKBKUAII-KBXCAEBGSA-N,LOXO-101,46189000.0,...,-4.415429,0.361754,1.688338,1.427239,1.427239,2.269618,2.269618,2.751650,-3.652360,-3.720239
4,12676,6,BRD-K74913225-001-14-0,DMSO,Compound,BR00116991,A07,HCRKCZRJWPKOAR-JTQLQIEISA-N,brinzolamide,68844.0,...,-4.527514,0.309408,2.686301,1.208430,1.208430,1.682038,1.682038,3.294929,-3.826002,-4.035504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11099,19447,333,NaN,DMSO,Compound,BR00117024,N22,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,DMSO,679.0,...,-0.687591,-0.169181,-0.044100,-0.825062,-0.825062,0.026758,0.026758,-0.193469,0.997390,0.790256
11100,19456,342,NaN,DMSO,Compound,BR00117024,O07,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,DMSO,679.0,...,0.143381,-0.413587,0.792243,-0.109585,-0.109585,-0.189539,-0.189539,-0.169685,-0.230320,-0.219103
11101,19467,353,NaN,DMSO,Compound,BR00117024,O18,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,DMSO,679.0,...,-0.370429,0.110826,0.593795,-0.166704,-0.166704,-0.283637,-0.283637,0.137650,-0.222451,-0.297309
11102,19478,364,NaN,DMSO,Compound,BR00117024,P05,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,DMSO,679.0,...,1.017101,-2.176102,1.777201,-0.389053,-0.389053,-0.274323,-0.274323,0.162904,0.188012,0.150115


In [13]:
dfnew.iloc[:,1:].to_csv('SLPP_seen_new.csv')          # After feature transformation
df_test1.to_csv('SLPP_seen_old.csv')                  # Before feature transformation

In [14]:
# Train_test pre-process Unseen test data
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
y_test=df_test2['Metadata_gene']
testX=df_test2.drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)
X_train = trainX
y_train = y_train.reset_index()
y_train=y_train['Metadata_gene'].tolist()

In [16]:
# Transformed Unseen test Data
transformed_data = np.dot(testX, eigvecs)
new_feature=pd.DataFrame(transformed_data)
testX2=df_test2.iloc[:,:df_test2.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
testX2=testX2.reset_index()
dfnew =pd.concat([testX2,new_feature],axis=1)

,index,Unnamed: 0,Metadata_broad_sample,Metadata_solvent,Metadata_experiment_type,Metadata_Plate,Metadata_Well,Metadata_InChIKey,Metadata_pert_iname,Metadata_pubchem_cid,...,290,291,292,293,294,295,296,297,298,299
0,4,4,BRD-K48278478-001-01-2,DMSO,Compound,BR00117016,A05,LOUPRKONTZGTKE-AFHBHXEDSA-N,quinine,94175.0,...,1.092467,0.500929,0.216295,0.362195,0.362195,0.151032,0.151032,0.431012,1.023097,0.970503
1,19,19,BRD-K91825936-001-03-9,DMSO,Compound,BR00117016,A20,XQYASZNUFDVMFH-CQSZACIVSA-N,ZK811752,512280.0,...,0.110996,0.929917,0.882559,-0.418358,-0.418358,-0.465443,-0.465443,0.746460,0.809494,0.488814
2,20,20,BRD-K38775274-001-22-1,DMSO,Compound,BR00117016,A21,UREBDLICKHMUKA-CXSFZGCWSA-N,dexamethasone,5743.0,...,-0.601985,3.874776,-2.073626,-1.232122,-1.232122,-0.811724,-0.811724,0.800959,3.035211,1.999204
3,24,24,BRD-K26439554-001-01-3,DMSO,Compound,BR00117016,B01,HFYPTENHTPNXGP-UHFFFAOYSA-N,H2L-5765834,1365700.0,...,0.200092,-0.271784,1.103198,0.271246,0.271246,-0.141582,-0.141582,0.186843,-0.361376,-0.155144
4,30,30,BRD-K06890060-001-09-1,DMSO,Compound,BR00117016,B07,NMKJFZCBCIUYHI-UHFFFAOYSA-N,alda-1,831630.0,...,0.496772,-0.612138,-0.256246,-0.569730,-0.569730,-0.304397,-0.304397,-0.253571,0.290003,0.311184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9051,18339,377,BRDN0001054361,NaN,CRISPR,BR00118045,P18,NaN,NaN,NaN,...,0.750643,-0.665028,0.201964,-1.348142,-1.348142,-0.162346,-0.162346,0.462769,-0.863486,-0.966065
9052,18340,378,BRDN0001481136,NaN,CRISPR,BR00118045,P19,NaN,NaN,NaN,...,0.038349,0.274298,0.363692,-0.727366,-0.727366,-0.349959,-0.349959,0.794445,-0.342594,-0.809589
9053,18342,380,BRDN0001484228,NaN,CRISPR,BR00118045,P21,NaN,NaN,NaN,...,-0.273884,-0.549777,1.446461,-0.195423,-0.195423,0.193776,0.193776,0.961136,-1.810151,-2.251023
9054,18343,381,BRDN0001487618,NaN,CRISPR,BR00118045,P22,NaN,NaN,NaN,...,0.811293,-1.044456,-0.028276,-1.004114,-1.004114,0.348762,0.348762,0.645620,-1.460919,-1.727635


In [17]:
dfnew.iloc[:,1:].to_csv('SLPP_unseen_new.csv')
df_test1.to_csv('SLPP_unseen_old.csv')    